In [ ]:
from System import *
from QuantConnect import *
from QuantConnect.Data.Consolidators import *
from QuantConnect.Data.Market import *
from QuantConnect.Orders import OrderStatus
from QuantConnect.Algorithm import QCAlgorithm
from QuantConnect.Indicators import *
import numpy as np
from datetime import timedelta, datetime
from datetime import date

class EMAMomentumUniverse(QCAlgorithm):
    
    def Initialize(self):
        # Define backtest window and portfolio cash
        self.SetStartDate(2014, 2, 1)
        self.SetEndDate(2020, 12, 1)
        self.SetCash(50000)
        # Add the assets to be fed into the algorithm and save the symbol objects (to be referred later)
        self.ticker=["SOXL"]
        for i in self.ticker:
            self.ewc_symbol = self.AddEquity(i, Resolution.Daily).Symbol
            self.Log(i)
        
        self.tick =["SOXS"]
        self.SetWarmup(TimeSpan.FromDays(30))
        for j in self.tick:
            self.ewa_symbol = self.AddEquity(j, Resolution.Daily).Symbol
            self.Log(j)
        
        #self.vti_symbol = self.AddEquity("VTI", Resolution.Daily).Symbol
        
        self.tic= ['VTI']
        for k in self.tic:
            self.vti_symbol = self.AddEquity("VTI", Resolution.Daily).Symbol
            self.Schedule.On(self.DateRules.EveryDay(self.vti_symbol), self.TimeRules.At(13, 59), self.SpecificTime)
        
            self.Log(k)
            
        #self.Schedule.On(self.DateRules.EveryDay(), self.TimeRules.At(13, 59), self.SpecificTime)
        
        
       
        
        # Define the objectives when going long or going short (long=buy) (short=sell)
         # Loop through our equities
        #for Symbol,symboll  in zip(self.ticker,self.tick):
        self.long_targets = [PortfolioTarget("SOXL", 0.5), PortfolioTarget("SOXS", -0.5)]
        
        #for i in self.ticker:
        #for Symbol,symboll  in zip(self.ticker,self.tick):
        self.short_targets = [PortfolioTarget("SOXL", 1.0), PortfolioTarget("SOXS", 0)]

        self.is_invested = None
        self.window = RollingWindow[TradeBar](8)
      
        self.Schedule.On(self.DateRules.EveryDay("VTI"), self.TimeRules.At(13, 59), self.SpecificTime)
        
    def SpecificTime(self):
        self.Log("SpecificTime: Fired at : {0}".format(self.Time))
    
    def OnData(self, data):
       
        self.ticker=["SOXL"]
        self.tick =["SOXS"]
            # for daily bars data is delivered at 00:00 of the day containing the closing price of the previous day (23:59:59)
        for Symbol,symboll in zip(self.ticker,self.tick):
            #if (not data.Bars.ContainsKey(self.ewc_symbol)) or (not data.Bars.ContainsKey(self.ewa_symbol)) or (not data.Bars.ContainsKey(self.ewc_symbol)):
            #    return
            if (not data.Bars.ContainsKey(Symbol)) or (not data.Bars.ContainsKey(symboll)):# or (not data.Bars.ContainsKey(symbolll)):
                return
            self.tic=["VTI"]
            for i in self.tick:
                self.window.Add(data[i])
          
            if not self.window.IsReady:
                return
    
            #serie = self.series.Current.Value
            currBar = self.window[0]
            currBarr= self.window[4]
            price= currBar.Close
            pricee=currBarr.Close
          
            # if it is not invested, see if there is an entry point
            current_time = self.Time
    
            #if (current_time.hour == 13 and current_time.minute == 59):
            if not self.is_invested:    
                # if our portfolio is bellow the lower band, enter long
                if price < pricee:
                    self.SetHoldings(self.long_targets)
    
                    self.Debug('Entering Long and short')
                    self.is_invested = 'long_short'
                else:
                    self.SetHoldings(self.short_targets)
                   # self.Liquidate()
                    self.Debug('Exiting Long')
                    self.is_invested = None
                    
                
            # if it is invested in something, check the exiting signal    
            if self.is_invested == 'long_short':
               if price > pricee:
                    self.SetHoldings(self.short_targets)
                   # self.Liquidate()
                    self.Debug('Exiting Long')
                    self.is_invested = None
                